## Imports

In [1]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline  

from sklearn import svm, linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../../../Utils/')

import models
from train import *
from metrics import * 
from SVC_Utils import *

#audio
import librosa as libr

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Python: 3.6.5 (default, Jul  6 2018, 19:12:46) 
[GCC 5.4.0 20160609]
Pytorch: 0.4.0


## NN Hyperparameters

In [2]:
batch_size = 128
lr = 0.001
k = 3

pretrained = True #run this with networks that have already been trained

transform_type = 'SFTF' #either STFT or MFCC  

if transform_type == 'SFTF':
    target_net_type = models.STFT_CNN_classifier
    shadow_net_type = models.STFT_CNN_classifier
    in_size = 94# 20 forMFCC,  94 for STFT
elif transform_type == 'MFCC':
    target_net_type = models.audio_CNN_classifier
    shadow_net_type = models.audio_CNN_classifier
    in_size = 20

## Load LibriSpeech data:

In [3]:
### Hyperparameters

n_seconds = 3
n_epochs = 25
sampling_rate = 16000
number_of_mels =128
data_set = 'all' #100 or 360 or all
if data_set == 100:
    all_data = ['train-clean-100']
elif data_set == 360:
    all_data = ['train-clean-360']
elif data_set == 1000:
    all_data = ['train-clean-100','train-clean-360','dev_clean','test_clean']
lr = 0.001

### Speech preprocessing

class tensorToMFCC:
    def __call__(self, y):
#         y = y.numpy()
        dims = y.shape
        y = libr.feature.melspectrogram(np.reshape(y, (dims[1],)), 16000, n_mels=number_of_mels,
                               fmax=8000)
        y = libr.feature.mfcc(S = libr.power_to_db(y))
        y = torch.from_numpy(y)                           
        return y.float()


class STFT:
    def __call__(self,y):
        dims = y.shape
        y = np.abs(libr.core.stft(np.reshape(y, (dims[1],))))
        y = torch.from_numpy(y).permute(1,0)
        return y.float()

if transform_type == 'SFTF':
    transform  = STFT() ## STFT or MFCC
elif transform_type == 'MFCC':
    transform  = tensorToMFCC()
    
### Data set

%load_ext autoreload
%autoreload 2
sys.path.insert(0, './../../../Utils')
from datasets import LibriSpeechDataset
from datasets import audio_preload_and_split

path = './../../../Classification_baselines/LibriSpeech/data/'

splits = [0.8, 0.2] #input fraction of data you want partitioned. Train, test fraction
attacking = 1 #0 for no attack, 1 for attack 1, 3 for attack 3
# attacking means data for a target & shadow network.
# This will also split "out data" from totally different speakers -- data none of the 
# other networks have seen, for training & testing the attack network. This will be
# an equivalent amount of data to the train split as defined about

if sum(splits) != 1:
    print('error: splits do not sum to 1.')


#Splits data into 2 sets of speakers for target & shadow network, into above defined train:test splits
dfs = Libri_preload_and_split(path,all_data,n_seconds,pad=False,cache=True,splits=splits, attacking = attacking)  

#target train & test
valid_sequence_train = LibriSpeechDataset(path, df = dfs[0], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)

valid_sequence_test = LibriSpeechDataset(path, df = dfs[1], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)

#shadow train & test
valid_sequence_train_shadow = LibriSpeechDataset(path, df = dfs[2], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)



valid_sequence_test_shadow = LibriSpeechDataset(path, df = dfs[3], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)


valid_sequence_out = LibriSpeechDataset(path, df = dfs[4], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)

valid_sequence_out_shadow = LibriSpeechDataset(path, df = dfs[5], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)

Initialising LibriSpeechDataset with minimum length = 3s and subsets = ['train-clean-100']
Finished indexing data. 27949 usable files found.
6039
1544
6279
1603
6210
6274
Finished splitting data.


In [27]:
batch_size = 32


# Loaders for data for target model & shadow model 
target_train_loader = DataLoader(valid_sequence_train,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )

target_out_loader = DataLoader(valid_sequence_out,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )

target_test_loader = DataLoader(valid_sequence_test,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

shadow_train_loader = DataLoader(valid_sequence_train_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )

shadow_out_loader = DataLoader(valid_sequence_out_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )

shadow_test_loader = DataLoader(valid_sequence_test_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


# Initialize/Train Targets
The model being attacked; if network, architecture can differ from that of shadow network.

In [5]:
#Initialize NN

#in_size defined above
n_hidden = 512
n_classes = valid_sequence_test.num_speakers
print(n_classes)

target_net = target_net_type(in_size,n_hidden,n_classes).to(device)
target_net.apply(models.weights_init)

target_loss = nn.CrossEntropyLoss()
target_optim = optim.Adam(target_net.parameters(), lr=lr)

69


In [6]:
#Train NN
if not pretrained:
    train(target_net, target_train_loader, target_test_loader, target_optim, target_loss, n_epochs, verbose = False) #classes = range(valid_sequence_test.num_speakers),
else:
    fnm = 'model_weights/CNN_voice_classifier'+str(data_set)+'_target_'+transform_type+str(n_epochs-1)+'.pth'
#     fnm = 'model_weights/CNN_voice_classifier100_target_'+str(epoch)+'.pth'
    chpt = torch.load(fnm)
    target_net.load_state_dict(chpt['state_dict'])

In [7]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        
epoch = 24

In [8]:
if not pretrained:

    save_checkpoint({
                'epoch': epoch,
                'arch': 'audio_CNN_classifier',
                'state_dict': target_net.state_dict(),
                'optimizer' : target_optim.state_dict(),
            }, False, filename = 'model_weights/CNN_voice_classifier'+str(data_set)+'_target_'+transform_type+str(epoch)+'.pth')

### Hold off on adapting SVM for now

In [9]:
# #Initialize SVM

# # #The stored baseline SVM was fit using all of CIFAR10 training data. To attack for membership inference, use 
# # #images not in CIFAR10 training data, or fit new classifiers/run source code with subset of CIFAR10.

# # '''
# # dir='../../../Classification_baselines/CIFAR10'
# # target_gen=load_svm(dir, gen=True)
# # target_maxacc=load_svm(dir, gen=False)
# # '''

# # #Training example targets on loaded CIFAR10 target subset:

# gen_svm=make_pipeline(PCA(n_components=180), MinMaxScaler(feature_range=(-1,1)), svm.SVC(C=10, gamma=.1, probability=True))
# maxacc_svm=make_pipeline(PCA(n_components=180), MinMaxScaler(feature_range=(-1,1)), svm.SVC(C=1, gamma=.01, probability=True))

# # sv_target_fit_loader = torch.utils.data.DataLoader(sv_cifar10_trainset, batch_size=target_train_sampler.__len__(), 
# #                                                    sampler=target_train_sampler, num_workers=1)


# tin, tout=load(target_train_loader)

# #Train SVM
# gen_svm.fit(tin, tout)
# maxacc_svm.fit(tin, tout)

In [10]:
# #evaluate SVM targets

# classes = range(n_classes)
# inp, outp=load(target_test_loader)

# print('SVM A (C=', gen_svm.get_params(deep=True)['svc__C'], ', gamma= ',
#       gen_svm.get_params(deep=True)['svc__gamma'], '): ')
# class_acc(gen_svm.predict_proba(inp), outp, classes)

# print('SVM B (C=', maxacc_svm.get_params(deep=True)['svc__C'], ', gamma= ',
#       maxacc_svm.get_params(deep=True)['svc__gamma'], '): ')
# class_acc(maxacc_svm.predict_proba(inp), outp, classes)

# Initialize/Train Shadow Model
Shadow model mimics the target network, emulating the target model's differences in prediction probabilities for samples in and out of its dataset. For this attack, only one shadow model is used. 

In [11]:
#Initialize models

n_classes = valid_sequence_test_shadow.num_speakers
print(n_classes)

#NN
shadow_net = shadow_net_type(in_size,n_hidden,n_classes).to(device)
shadow_net.apply(models.weights_init)

shadow_loss = nn.CrossEntropyLoss()
shadow_optim = optim.Adam(shadow_net.parameters(), lr=lr)

#SVM
# shadowinputs, shadowtargets=load(sv_shadow_train_loader)
# shadow_svm=make_pipeline(PCA(n_components=180), MinMaxScaler(feature_range=(-1,1)), 
#                          svm.SVC(C=1, gamma=.1, probability=True))

69


In [12]:
# print(np.shape(shadowinputs))
# shadowtargets[5]

In [13]:
#Train NN

# below commented code is for comparison during debugging
# target_net = target_net_type(in_size,n_hidden,n_classes).to(device)
# target_net.apply(models.weights_init)

# target_loss = nn.CrossEntropyLoss()
# target_optim = optim.Adam(target_net.parameters(), lr=lr)

# train(target_net, target_train_loader, target_test_loader, target_optim, target_loss, n_epochs, verbose = False) #classes = range(valid_sequence_test.num_speakers),

if not pretrained:
    train(shadow_net, shadow_train_loader, shadow_test_loader, shadow_optim, shadow_loss, n_epochs, verbose = False)
else:
    fnm = 'model_weights/CNN_voice_classifier'+str(data_set)+'_shadow_'+transform_type+str(n_epochs-1)+'.pth'
#     fnm = 'model_weights/CNN_voice_classifier100_target_'+str(epoch)+'.pth'
    chpt = torch.load(fnm)
    shadow_net.load_state_dict(chpt['state_dict'])

In [14]:
epoch = 24
save_checkpoint({
            'epoch': epoch,
            'arch': 'CNN_voice_classifier',
            'state_dict': shadow_net.state_dict(),
            'optimizer' : shadow_optim.state_dict(),
        }, False, filename = 'model_weights/CNN_voice_classifier'+str(data_set)+'_shadow_'+transform_type+str(epoch)+'.pth')

In [15]:
#Train SVM
# shadow_svm.fit(shadowinputs, shadowtargets)

# Initialize Attack Model
A binary classifier to determine membership. 

In [16]:
#Creates two attack nets for comparison.

# attack_net_svm = models.mlleaks_mlp(n_in=k).to(device)
# attack_net_svm.apply(models.weights_init)

attack_net_nn = models.mlleaks_mlp(n_in=k).to(device)
attack_net_nn.apply(models.weights_init)

attack_loss = nn.BCEWithLogitsLoss() #this one works
# attack_loss = nn.BCELoss() # this one doesn't work 
# attack_optim_svm= optim.Adam(attack_net_svm.parameters(), lr=lr)
attack_optim_nn= optim.Adam(attack_net_nn.parameters(), lr=lr)

In [17]:
#Trains SVM attack model
# train_attacker(attack_net_svm, shadow_svm, shadow_train_loader, shadow_out_loader, attack_optim_svm, attack_loss, n_epochs=2, k=k)

In [25]:
pretrained = False

In [28]:
#Trains NN attack model
attack_loss = nn.BCEWithLogitsLoss()
n_epochs_attack = 50

if not pretrained:
    train_attacker(attack_net_nn, shadow_net, shadow_train_loader, shadow_out_loader, attack_optim_nn, 
                   attack_loss, n_epochs=n_epochs_attack, k=k)
    
else:
    fnm = 'model_weights/CNN_voice_classifier'+str(data_set)+'_attack_'+transform_type+str(24)+'.pth'
#     fnm = 'model_weights/CNN_voice_classifier100_target_'+str(epoch)+'.pth'
    chpt = torch.load(fnm)
    attack_net_nn.load_state_dict(chpt['state_dict'])

#original:
# train_attacker(attack_net_nn, shadow_net, shadow_train_loader, shadow_out_loader, attack_optim_nn, attack_loss, n_epochs=50, k=k)

[0/50][0/196] loss = 0.83, accuracy = 50.00
[0/50][1/196] loss = 0.82, accuracy = 50.00
[0/50][2/196] loss = 0.81, accuracy = 50.00
[0/50][3/196] loss = 0.80, accuracy = 50.00
[0/50][4/196] loss = 0.79, accuracy = 50.00
[0/50][5/196] loss = 0.78, accuracy = 50.00
[0/50][6/196] loss = 0.78, accuracy = 50.00
[0/50][7/196] loss = 0.77, accuracy = 50.00
[0/50][8/196] loss = 0.76, accuracy = 50.00
[0/50][9/196] loss = 0.76, accuracy = 50.00
[0/50][10/196] loss = 0.75, accuracy = 50.00
[0/50][11/196] loss = 0.74, accuracy = 50.00
[0/50][12/196] loss = 0.74, accuracy = 50.00
[0/50][13/196] loss = 0.73, accuracy = 50.00
[0/50][14/196] loss = 0.73, accuracy = 50.00
[0/50][15/196] loss = 0.72, accuracy = 50.00
[0/50][16/196] loss = 0.72, accuracy = 50.00
[0/50][17/196] loss = 0.72, accuracy = 50.00
[0/50][18/196] loss = 0.71, accuracy = 50.00
[0/50][19/196] loss = 0.71, accuracy = 50.00
[0/50][20/196] loss = 0.71, accuracy = 50.00
[0/50][21/196] loss = 0.71, accuracy = 50.00
[0/50][22/196] loss 

[0/50][181/196] loss = 0.67, accuracy = 65.63
[0/50][182/196] loss = 0.66, accuracy = 65.69
[0/50][183/196] loss = 0.67, accuracy = 65.74
[0/50][184/196] loss = 0.66, accuracy = 65.80
[0/50][185/196] loss = 0.67, accuracy = 65.85
[0/50][186/196] loss = 0.67, accuracy = 65.88
[0/50][187/196] loss = 0.67, accuracy = 65.95
[0/50][188/196] loss = 0.67, accuracy = 65.96
[0/50][189/196] loss = 0.66, accuracy = 66.03
[0/50][190/196] loss = 0.67, accuracy = 66.08
[0/50][191/196] loss = 0.66, accuracy = 66.15
[0/50][192/196] loss = 0.67, accuracy = 66.20
[0/50][193/196] loss = 0.66, accuracy = 66.25
[0/50][194/196] loss = 0.67, accuracy = 66.29
[0/50][195/196] loss = 0.67, accuracy = 66.33
[1/50][0/196] loss = 0.66, accuracy = 71.88
[1/50][1/196] loss = 0.66, accuracy = 75.00
[1/50][2/196] loss = 0.66, accuracy = 76.56
[1/50][3/196] loss = 0.67, accuracy = 76.17
[1/50][4/196] loss = 0.66, accuracy = 75.62
[1/50][5/196] loss = 0.66, accuracy = 75.52
[1/50][6/196] loss = 0.66, accuracy = 76.12
[1

[1/50][167/196] loss = 0.60, accuracy = 75.62
[1/50][168/196] loss = 0.64, accuracy = 75.62
[1/50][169/196] loss = 0.61, accuracy = 75.67
[1/50][170/196] loss = 0.61, accuracy = 75.69
[1/50][171/196] loss = 0.61, accuracy = 75.71
[1/50][172/196] loss = 0.61, accuracy = 75.73
[1/50][173/196] loss = 0.65, accuracy = 75.68
[1/50][174/196] loss = 0.64, accuracy = 75.67
[1/50][175/196] loss = 0.63, accuracy = 75.65
[1/50][176/196] loss = 0.63, accuracy = 75.64
[1/50][177/196] loss = 0.62, accuracy = 75.63
[1/50][178/196] loss = 0.63, accuracy = 75.62
[1/50][179/196] loss = 0.62, accuracy = 75.63
[1/50][180/196] loss = 0.64, accuracy = 75.60
[1/50][181/196] loss = 0.61, accuracy = 75.62
[1/50][182/196] loss = 0.62, accuracy = 75.61
[1/50][183/196] loss = 0.62, accuracy = 75.60
[1/50][184/196] loss = 0.62, accuracy = 75.62
[1/50][185/196] loss = 0.65, accuracy = 75.55
[1/50][186/196] loss = 0.64, accuracy = 75.53
[1/50][187/196] loss = 0.63, accuracy = 75.52
[1/50][188/196] loss = 0.63, accur

[2/50][152/196] loss = 0.57, accuracy = 76.21
[2/50][153/196] loss = 0.58, accuracy = 76.20
[2/50][154/196] loss = 0.57, accuracy = 76.19
[2/50][155/196] loss = 0.59, accuracy = 76.15
[2/50][156/196] loss = 0.59, accuracy = 76.14
[2/50][157/196] loss = 0.57, accuracy = 76.17
[2/50][158/196] loss = 0.56, accuracy = 76.21
[2/50][159/196] loss = 0.56, accuracy = 76.21
[2/50][160/196] loss = 0.58, accuracy = 76.22
[2/50][161/196] loss = 0.64, accuracy = 76.15
[2/50][162/196] loss = 0.59, accuracy = 76.15
[2/50][163/196] loss = 0.56, accuracy = 76.20
[2/50][164/196] loss = 0.54, accuracy = 76.23
[2/50][165/196] loss = 0.54, accuracy = 76.27
[2/50][166/196] loss = 0.60, accuracy = 76.24
[2/50][167/196] loss = 0.58, accuracy = 76.21
[2/50][168/196] loss = 0.57, accuracy = 76.22
[2/50][169/196] loss = 0.58, accuracy = 76.22
[2/50][170/196] loss = 0.52, accuracy = 76.26
[2/50][171/196] loss = 0.56, accuracy = 76.27
[2/50][172/196] loss = 0.61, accuracy = 76.23
[2/50][173/196] loss = 0.60, accur

[3/50][138/196] loss = 0.53, accuracy = 76.33
[3/50][139/196] loss = 0.55, accuracy = 76.28
[3/50][140/196] loss = 0.58, accuracy = 76.23
[3/50][141/196] loss = 0.52, accuracy = 76.25
[3/50][142/196] loss = 0.51, accuracy = 76.28
[3/50][143/196] loss = 0.48, accuracy = 76.33
[3/50][144/196] loss = 0.46, accuracy = 76.41
[3/50][145/196] loss = 0.57, accuracy = 76.39
[3/50][146/196] loss = 0.54, accuracy = 76.36
[3/50][147/196] loss = 0.54, accuracy = 76.33
[3/50][148/196] loss = 0.51, accuracy = 76.37
[3/50][149/196] loss = 0.52, accuracy = 76.39
[3/50][150/196] loss = 0.51, accuracy = 76.41
[3/50][151/196] loss = 0.49, accuracy = 76.43
[3/50][152/196] loss = 0.53, accuracy = 76.42
[3/50][153/196] loss = 0.52, accuracy = 76.43
[3/50][154/196] loss = 0.58, accuracy = 76.40
[3/50][155/196] loss = 0.56, accuracy = 76.37
[3/50][156/196] loss = 0.47, accuracy = 76.43
[3/50][157/196] loss = 0.52, accuracy = 76.42
[3/50][158/196] loss = 0.52, accuracy = 76.42
[3/50][159/196] loss = 0.52, accur

...

[46/50][61/196] loss = 0.36, accuracy = 84.40
[46/50][62/196] loss = 0.41, accuracy = 84.35
[46/50][63/196] loss = 0.40, accuracy = 84.38
[46/50][64/196] loss = 0.36, accuracy = 84.40
[46/50][65/196] loss = 0.36, accuracy = 84.42
[46/50][66/196] loss = 0.34, accuracy = 84.44
[46/50][67/196] loss = 0.58, accuracy = 84.44
[46/50][68/196] loss = 0.33, accuracy = 84.47
[46/50][69/196] loss = 0.66, accuracy = 84.53
[46/50][70/196] loss = 0.30, accuracy = 84.57
[46/50][71/196] loss = 0.32, accuracy = 84.59
[46/50][72/196] loss = 0.51, accuracy = 84.44
[46/50][73/196] loss = 0.24, accuracy = 84.54
[46/50][74/196] loss = 0.97, accuracy = 84.35
[46/50][75/196] loss = 0.30, accuracy = 84.42
[46/50][76/196] loss = 0.40, accuracy = 84.42
[46/50][77/196] loss = 0.30, accuracy = 84.46
[46/50][78/196] loss = 0.37, accuracy = 84.43
[46/50][79/196] loss = 0.30, accuracy = 84.47
[46/50][80/196] loss = 0.24, accuracy = 84.59
[46/50][81/196] loss = 0.27, accuracy = 84.66
[46/50][82/196] loss = 0.41, accur

[47/50][42/196] loss = 0.60, accuracy = 85.54
[47/50][43/196] loss = 0.34, accuracy = 85.55
[47/50][44/196] loss = 0.34, accuracy = 85.56
[47/50][45/196] loss = 0.33, accuracy = 85.56
[47/50][46/196] loss = 0.38, accuracy = 85.51
[47/50][47/196] loss = 0.32, accuracy = 85.51
[47/50][48/196] loss = 0.41, accuracy = 85.40
[47/50][49/196] loss = 0.35, accuracy = 85.38
[47/50][50/196] loss = 0.47, accuracy = 85.20
[47/50][51/196] loss = 0.33, accuracy = 85.25
[47/50][52/196] loss = 0.46, accuracy = 85.11
[47/50][53/196] loss = 0.29, accuracy = 85.19
[47/50][54/196] loss = 0.35, accuracy = 85.11
[47/50][55/196] loss = 0.38, accuracy = 85.10
[47/50][56/196] loss = 0.30, accuracy = 85.12
[47/50][57/196] loss = 0.53, accuracy = 85.08
[47/50][58/196] loss = 0.37, accuracy = 85.06
[47/50][59/196] loss = 0.29, accuracy = 85.13
[47/50][60/196] loss = 0.40, accuracy = 85.04
[47/50][61/196] loss = 0.35, accuracy = 85.06
[47/50][62/196] loss = 0.26, accuracy = 85.17
[47/50][63/196] loss = 0.32, accur

[48/50][23/196] loss = 0.64, accuracy = 85.48
[48/50][24/196] loss = 0.31, accuracy = 85.50
[48/50][25/196] loss = 0.33, accuracy = 85.46
[48/50][26/196] loss = 0.29, accuracy = 85.59
[48/50][27/196] loss = 0.53, accuracy = 85.77
[48/50][28/196] loss = 0.43, accuracy = 85.51
[48/50][29/196] loss = 0.34, accuracy = 85.52
[48/50][30/196] loss = 0.24, accuracy = 85.79
[48/50][31/196] loss = 0.43, accuracy = 85.60
[48/50][32/196] loss = 0.40, accuracy = 85.61
[48/50][33/196] loss = 0.35, accuracy = 85.52
[48/50][34/196] loss = 0.63, accuracy = 85.36
[48/50][35/196] loss = 0.36, accuracy = 85.24
[48/50][36/196] loss = 0.44, accuracy = 85.18
[48/50][37/196] loss = 0.49, accuracy = 84.87
[48/50][38/196] loss = 0.74, accuracy = 84.74
[48/50][39/196] loss = 0.32, accuracy = 84.77
[48/50][40/196] loss = 0.42, accuracy = 84.79
[48/50][41/196] loss = 0.38, accuracy = 84.78
[48/50][42/196] loss = 0.33, accuracy = 84.81
[48/50][43/196] loss = 0.32, accuracy = 84.84
[48/50][44/196] loss = 0.30, accur

[49/50][4/196] loss = 0.38, accuracy = 84.06
[49/50][5/196] loss = 0.31, accuracy = 84.64
[49/50][6/196] loss = 0.57, accuracy = 85.49
[49/50][7/196] loss = 0.38, accuracy = 85.16
[49/50][8/196] loss = 0.37, accuracy = 84.90
[49/50][9/196] loss = 0.39, accuracy = 84.53
[49/50][10/196] loss = 0.68, accuracy = 84.09
[49/50][11/196] loss = 0.33, accuracy = 84.38
[49/50][12/196] loss = 0.32, accuracy = 84.50
[49/50][13/196] loss = 0.65, accuracy = 83.93
[49/50][14/196] loss = 0.36, accuracy = 83.96
[49/50][15/196] loss = 0.46, accuracy = 83.98
[49/50][16/196] loss = 0.33, accuracy = 84.10
[49/50][17/196] loss = 0.35, accuracy = 84.29
[49/50][18/196] loss = 0.35, accuracy = 84.38
[49/50][19/196] loss = 0.34, accuracy = 84.45
[49/50][20/196] loss = 0.38, accuracy = 84.23
[49/50][21/196] loss = 0.60, accuracy = 84.09
[49/50][22/196] loss = 0.30, accuracy = 84.17
[49/50][23/196] loss = 0.41, accuracy = 84.05
[49/50][24/196] loss = 0.32, accuracy = 84.19
[49/50][25/196] loss = 0.29, accuracy = 

[49/50][181/196] loss = 0.29, accuracy = 85.13
[49/50][182/196] loss = 0.38, accuracy = 85.10
[49/50][183/196] loss = 0.31, accuracy = 85.10
[49/50][184/196] loss = 0.36, accuracy = 85.09
[49/50][185/196] loss = 0.26, accuracy = 85.13
[49/50][186/196] loss = 0.43, accuracy = 85.09
[49/50][187/196] loss = 0.29, accuracy = 85.11
[49/50][188/196] loss = 0.33, accuracy = 85.11
[49/50][189/196] loss = 0.50, accuracy = 85.13
[49/50][190/196] loss = 0.40, accuracy = 85.10
[49/50][191/196] loss = 0.43, accuracy = 85.07
[49/50][192/196] loss = 0.33, accuracy = 85.07
[49/50][193/196] loss = 0.46, accuracy = 85.04
[49/50][194/196] loss = 0.37, accuracy = 85.02
[49/50][195/196] loss = 0.43, accuracy = 85.01


In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        

save_checkpoint({
            'epoch': epoch,
            'arch': 'CNN_voice_classifier',
            'state_dict': attack_net_nn.state_dict(),
            'optimizer' : attack_optim_nn.state_dict(),
        }, False, filename = 'model_weights/CNN_voice_classifier'+str(data_set)+'_attack_'+transform_type+str(n_epochs_attack)+'.pth')

# Evaluate Attack Nets
How well the trained attack models classify a sample as in or out of a target model's training dataset, and how performance is affected by target hyperparameters and which models attack which targets.

In [ ]:
#attack net trained on svm shadow model on svm target, C=10, gamma=.1
# eval_attack_net(attack_net_svm, gen_svm, sv_target_train_loader, sv_target_out_loader, k)

In [ ]:
#attack net trained on svm shadow model on svm target, C=1, gamma=.01
# eval_attack_net(attack_net_svm, maxacc_svm, sv_target_train_loader, sv_target_out_loader, k)

In [ ]:
#attack net trained on nn shadow model on nn target
eval_attack_net(attack_net_nn, target_net, target_train_loader, target_out_loader, k)

In [ ]:
#attack net trained on nn shadow model on svm target, C=10, gamma=.1
# eval_attack_net(attack_net_nn, gen_svm, sv_target_train_loader, sv_target_out_loader, k)

In [29]:
#table with 

#baselines
columns = ['Transform','Training epochs', '# speakers','Train accuracy', 'Test accuracy', 'Attack type', 'Precision','Recall']

# do this for 10 & 100 speakers
# .2 S & 3 S
#sufficient training and over-training

#manual data: 
df = pd.DataFrame(columns = columns)
#Attack 1:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,1,0.89,0.90] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,1,0.88,0.91] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,1,0.89,0.92] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,1,0.85,0.86] 

#Attack 3 w/max data: 
df.loc[len(df)] = ['STFT',25,139.0,.9985,.9073,3,.81,.90] 
df.loc[len(df)] = ['STFT',50,511.0,.9942,.9057,3,.84,.87] 
df.loc[len(df)] = ['MFCC',25,139.0,.9969,.9136,3,.82,.92] 
df.loc[len(df)] = ['MFCC',25,511.0,.9960,.9321,3,0.83,0.93]


#Attack 3 on Attack1 models:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,3,0.84,0.95] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,3,0.84,0.94] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,3,0.81,0.97] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,3,0.81,0.90] 

df['Training epochs'] =df['Training epochs'].astype(float)
df['Attack type'] =df['Attack type'].astype(float)

#style table
import seaborn as sns

cg = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette("magenta", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
orr = sns.light_palette("orange", as_cmap=True)
gr = sns.light_palette("gray", as_cmap=True)

# df.style.bar(subset=['Train accuracy', 'Test accuracy'], align='mid', color=['#d65f5f', '#5fba7d'])
s = df.style.\
    background_gradient(cmap=cg,subset=['Train accuracy', 'Test accuracy']).\
    background_gradient(cmap=bl,subset=['Precision', 'Recall']).\
    background_gradient(cmap=orr,subset=['Training epochs']).\
    background_gradient(cmap=gr,subset=['Attack type']).\
    background_gradient(cmap=cm,subset=['# speakers']).\
    format({"Train accuracy": "{:.2%}","Test accuracy": "{:.2%}"}).\
    hide_index().\
    set_properties(**{'font-size': "16pt",'column-size':"24pt",'width': '100px'})

s

Transform,Training epochs,# speakers,Train accuracy,Test accuracy,Attack type,Precision,Recall
MFCC,25,69,99.94%,96.32%,1,0.89,0.9
MFCC,25,255,99.61%,94.43%,1,0.88,0.91
STFT,25,69,99.89%,94.51%,1,0.89,0.92
STFT,25,255,99.58%,91.81%,1,0.85,0.86
STFT,25,139,99.85%,90.73%,3,0.81,0.9
STFT,50,511,99.42%,90.57%,3,0.84,0.87
MFCC,25,139,99.69%,91.36%,3,0.82,0.92
MFCC,25,511,99.60%,93.21%,3,0.83,0.93
MFCC,25,69,99.94%,96.32%,3,0.84,0.95
MFCC,25,255,99.61%,94.43%,3,0.84,0.94
